In [1]:
%pip install xgboost
%pip install streamlit
%pip install pandas
%pip install python-dotenv
%pip install seaborn
%pip install matplotlib
%pip install pyodbc
%pip install numpy
%pip install xlrd
%pip install openpyxl
%pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 475.1 kB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 276.7 kB/s eta 0:00:0000:0100:01
  Using cached altair-5.1.2-py3-none-any.whl (516 kB)
  Using cached cachetools-5.3.1-py3-none-any.whl (9.3 kB)
  Using cached click-8.1.7-py3-none-any.whl (97 kB)
  Using cached importlib_metadata-6.8.0-py3-none-any.whl (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.4/409.4 kB 454.9 kB/s eta 0:00:0000:0100:01
  Using cached pyarrow-13.0.0-cp311-cp311-macosx_10_14_x86_64.whl (25.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.8/239.8 kB 117.9 kB/s eta 0:00:0000:0100:01
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached typing_extensions-4.8.0-py3-none-any.whl (31 kB)
  Using cached validators-0.22.0-

In [2]:
# importing needed libraries

import streamlit as st
from dotenv import dotenv_values
import pyodbc
import os
import pickle
import xgboost as xgb
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn import metrics

from math import sqrt
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, explained_variance_score


from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import norm, chi2_contingency


from sklearn.impute import  SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV




import warnings
warnings.filterwarnings('ignore')

In [5]:
dat = pd.read_csv("ts_data.csv")

In [6]:
dat

,holiday,locale,transferred,sales,onpromotion
0,Holiday,National,False,2511.618999,0.0
1,Not Holiday,Not Holiday,False,496092.417944,0.0
2,Not Holiday,Not Holiday,False,361461.231124,0.0
3,Not Holiday,Not Holiday,False,354459.677093,0.0
4,Work Day,National,False,477350.121229,0.0
...,...,...,...,...,...
1698,Transfer,National,False,826373.722022,14179.0
1699,Not Holiday,Not Holiday,False,792630.535079,8312.0
1700,Not Holiday,Not Holiday,False,865639.677471,9283.0
1701,Not Holiday,Not Holiday,False,760922.406081,8043.0


In [7]:
dat.tail()

,holiday,locale,transferred,sales,onpromotion
1698,Transfer,National,False,826373.722022,14179.0
1699,Not Holiday,Not Holiday,False,792630.535079,8312.0
1700,Not Holiday,Not Holiday,False,865639.677471,9283.0
1701,Not Holiday,Not Holiday,False,760922.406081,8043.0
1702,Holiday,Local,False,762661.935939,10605.0


In [8]:
#from sklearn.preprocessing import StandardScaler

#scaler = StandardScaler()

# Example: Scaling numerical columns 'numerical_column_1' and 'numerical_column_2'
#dat[['holiday_type_Additional', 'holiday_type_Bridge', 'holiday_type_Event']] = scaler.fit_transform(dat[['holiday_type_Additional', 'holiday_type_Bridge', 'holiday_type_Event']])


In [9]:
dat.head()

,holiday,locale,transferred,sales,onpromotion
0,Holiday,National,False,2511.618999,0.0
1,Not Holiday,Not Holiday,False,496092.417944,0.0
2,Not Holiday,Not Holiday,False,361461.231124,0.0
3,Not Holiday,Not Holiday,False,354459.677093,0.0
4,Work Day,National,False,477350.121229,0.0


In [10]:
unique_holidays = dat['holiday'].unique()
print(unique_holidays)

['Holiday' 'Not Holiday' 'Work Day' 'Additional' 'Event' 'Transfer'
 'Bridge']


In [11]:
unique_holidays = dat['locale'].unique()
print(unique_holidays)

['National' 'Not Holiday' 'Local' 'Regional']


In [12]:
unique_holidays = dat['transferred'].unique()
print(unique_holidays)

[False  True]


In [13]:
# Encode the dataset first
df_encoded = pd.get_dummies(dat, columns=["holiday", "locale", "transferred"])

# Then split the data
X = df_encoded.drop('sales', axis=1)
y = df_encoded['sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


print(X_train)


      onpromotion  holiday_Additional  holiday_Bridge  holiday_Event  \
1196       5677.0               False           False          False   
422           0.0               False           False          False   
1668      20226.0               False           False          False   
575         673.0               False           False          False   
548         302.0               False           False           True   
...           ...                 ...             ...            ...   
1130       2791.0               False           False          False   
1294       5265.0               False           False          False   
860        1340.0               False           False          False   
1459      15863.0               False           False          False   
1126       3075.0               False           False          False   

      holiday_Holiday  holiday_Not Holiday  holiday_Transfer  \
1196            False                 True             False   
422    

In [14]:
X_train.columns

Index(['onpromotion', 'holiday_Additional', 'holiday_Bridge', 'holiday_Event',
       'holiday_Holiday', 'holiday_Not Holiday', 'holiday_Transfer',
       'holiday_Work Day', 'locale_Local', 'locale_National',
       'locale_Not Holiday', 'locale_Regional', 'transferred_False',
       'transferred_True'],
      dtype='object')

In [15]:
# Create and fit a Random Forest Regressor model

rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)

RandomForestRegressor()

In [16]:
# Create and fit an XGBoost Regressor model

xgbr = XGBRegressor()
xgbr.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [17]:
# Model Selection

selected_model = st.selectbox("Select a Model", ["XGBoost Regressor", "Random Forest Regressor"])

if selected_model == "XGBoost Regressor":
    model = xgbr
else:
    model = rfr


2023-10-15 13:02:11.953 
  command:

    streamlit run /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [18]:
# Calculate accuracy

model_acc = model.score(X_test, y_test) * 100

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import numpy as np

# Encoding and splitting the dataset
df_encoded = pd.get_dummies(dat, columns=["holiday", "locale", "transferred"])
X = df_encoded.drop('sales', axis=1)
y = df_encoded['sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Train XGBRegressor
model = XGBRegressor()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R2: {r2}")


MAE: 147570.53567409352
MSE: 40681658445.701
RMSE: 201696.94704110172
R2: 0.3981828499629252


In [20]:
# Make predictions

y_pred = model.predict(X_test)

In [21]:

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R2: {r2}")

MAE: 147570.53567409352
MSE: 40681658445.701
RMSE: 201696.94704110172
R2: 0.3981828499629252


In [22]:


# Ensure that both actual and predicted values are positive for RMSLE calculation
y_test_xgb = np.maximum(y_test, 0)
y_pred= np.maximum(y_pred, 0)

# Calculate RMSLE for XGBoost
msle_xgb = mean_squared_log_error(y_test_xgb, y_pred)
rmsle_xgb = np.sqrt(msle_xgb)


In [23]:
# Ensure that both actual and predicted values are positive for RMSLE calculation
y_test_xgb = np.maximum(y_test, 0)
y_pred = np.maximum(y_pred, 0)

# Calculate RMSLE for XGBoost
msle_xgb = mean_squared_log_error(y_test_xgb, y_pred)
rmsle = np.sqrt(msle_xgb)


In [24]:
# metric dictionary to store the results

metrics_dict = {
   "Metric": ["Mean Absolute Error (MAE)", "Mean Squared Error (MSE)", "Root Mean Squared Error (RMSE)",
              "R-squared (R2) Score", "Root Mean Squared Logarithmic Error (RMSLE)"],
   "Value": [round(mae, 2), round(mse, 5), round(rmse, 5), round(r2, 5), round(rmsle, 5)]
}
metrics_df = pd.DataFrame(metrics_dict)

In [25]:
metrics_df

,Metric,Value
0,Mean Absolute Error (MAE),1.475705e+05
1,Mean Squared Error (MSE),4.068166e+10
2,Root Mean Squared Error (RMSE),2.016969e+05
3,R-squared (R2) Score,3.981800e-01
4,Root Mean Squared Logarithmic Error (RMSLE),3.540100e-01


In [26]:
y_pred

array([ 630917.1 ,  755143.8 ,  610196.56,  401686.66,  836816.8 ,
        731227.2 ,  420997.84,  787319.9 ,  368504.34,  734181.9 ,
        420997.84,  489870.5 ,  420997.84,  777784.5 ,  420997.84,
        499751.47,  589183.8 ,  491011.44,  470380.12,  714435.94,
        938275.94,  710066.06,  717873.  ,  697020.4 ,  401686.66,
        470380.12,  420997.84,  470380.12,  570439.75,  420997.84,
        672872.6 ,  810083.9 ,  801124.5 ,  420997.84,  420997.84,
        595907.44,  714659.06,  666714.4 ,  595907.44,  908653.94,
        752009.4 ,  420997.84,  420997.84,  482754.06,  420997.84,
        816445.8 ,  895140.8 ,  420997.84,  447746.75,  944624.06,
        556363.94,  801124.5 ,  481554.88,  838718.1 ,  420997.84,
        420997.84,  420997.84,  420997.84,  888178.25,  420997.84,
        420997.84,  472609.7 ,  663875.4 ,  385600.84,  675680.06,
        717873.  ,  420997.84,  813402.75,  491011.44,  420997.84,
        532460.56,  720101.  ,  908653.94,  729119.6 ,  420997

#### Machine Learning Models

 1. Ridge regression Model


In [27]:
# Define the parameter grid for Ridge Regression
param_grid_ridge = {'alpha': [0.01, 0.1, 1.0, 10.0]}

# Create the Ridge regression model
ridge = Ridge()

# Perform grid search with cross-validation for Ridge Regression
grid_search_ridge = GridSearchCV(estimator=ridge, param_grid=param_grid_ridge, scoring='neg_mean_squared_error', cv=5)
grid_search_ridge.fit(X_train, y_train)

# Get the best hyperparameters for Ridge Regression
best_alpha = grid_search_ridge.best_params_['alpha']

# Train the final Ridge Regression model with the best hyperparameters
final_ridge = Ridge(alpha=best_alpha)
final_ridge.fit(X_train, y_train)

# Use the Ridge Regression model to make predictions on the test data
y_pred_ridge = final_ridge.predict(X_test)

# Ensure that both actual and predicted values are positive for RMSLE calculation
y_test_ridge = np.maximum(y_test, 0)
y_pred_ridge = np.maximum(y_pred_ridge, 0)
# Calculate the mean squared logarithmic error (MSLE) for Ridge Regression
msle_ridge = mean_squared_log_error(y_test_ridge, y_pred_ridge)

# Calculate the RMSLE for Ridge Regression
rmsle_ridge = np.sqrt(msle_ridge)


 2. Random Forest Model

In [28]:
# Define the parameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create the Random Forest Regressor model
rf = RandomForestRegressor()

# Perform grid search with cross-validation for Random Forest
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, scoring='neg_mean_squared_error', cv=5)
grid_search_rf.fit(X_train, y_train)

# Get the best hyperparameters for Random Forest
best_params_rf = grid_search_rf.best_params_

# Train the final Random Forest model with the best hyperparameters
final_rf = RandomForestRegressor(**best_params_rf)
final_rf.fit(X_train, y_train)

# Use the Random Forest model to make predictions on the test data
y_pred_rf = final_rf.predict(X_test)

# Ensure that both actual and predicted values are positive for RMSLE calculation
y_test_rf = np.maximum(y_test, 0)
y_pred_rf = np.maximum(y_pred_rf, 0)

# Calculate the mean squared logarithmic error (MSLE) for Random Forest
msle_rf = mean_squared_log_error(y_test_rf, y_pred_rf)

# Calculate the RMSLE for Random Forest
rmsle_rf = np.sqrt(msle_rf)
print("Random Forest RMSLE:", rmsle_rf)

Random Forest RMSLE: 0.3457168683662422


In [29]:
# 2. Random Forest
rf = RandomForestRegressor()
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, scoring='neg_mean_squared_error', cv=5)
grid_search_rf.fit(X_train, y_train)
y_pred_rf = grid_search_rf.best_estimator_.predict(X_test)
rmsle_rf = np.sqrt(mean_squared_log_error(np.maximum(y_test, 0), np.maximum(y_pred_rf, 0)))

 3. K Neighbors Regressor Model

In [30]:
# Define the parameter grid for K Neighbors Regressor
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

# Create the K Neighbors Regressor model
knn = KNeighborsRegressor()

# Perform grid search with cross-validation for K Neighbors Regressor
grid_search_knn = GridSearchCV(estimator=knn, param_grid=param_grid_knn, scoring='neg_mean_squared_error', cv=5)
grid_search_knn.fit(X_train, y_train)

# Get the best hyperparameters for K Neighbors Regressor
best_params_knn = grid_search_knn.best_params_

# Train the final K Neighbors Regressor model with the best hyperparameters
final_knn = KNeighborsRegressor(**best_params_knn)
final_knn.fit(X_train, y_train)

# Use the K Neighbors Regressor model to make predictions on the test data
y_pred_knn = final_knn.predict(X_test)

# Ensure that both actual and predicted values are positive for RMSLE calculation
y_test_knn = np.maximum(y_test, 0)
y_pred_knn = np.maximum(y_pred_knn, 0)

# Calculate the mean squared logarithmic error (MSLE) for K Neighbors Regressor
msle_knn = mean_squared_log_error(y_test_knn, y_pred_knn)

# Calculate the RMSLE for K Neighbors Regressor
rmsle_knn = np.sqrt(msle_knn)


4. XGBoost Model

In [31]:
# Define the parameter grid for XGBoost
param_grid_xgb = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'min_child_weight': [1, 2, 3]
}
# Create the XGBoost model
xgb_model = xgb.XGBRegressor()

# Perform grid search with cross-validation for XGBoost
grid_search_xgb = GridSearchCV(estimator=xgb_model, param_grid=param_grid_xgb, scoring='neg_mean_squared_error', cv=5)
grid_search_xgb.fit(X_train, y_train)

# Get the best hyperparameters for XGBoost
best_params_xgb = grid_search_xgb.best_params_

# Train the final XGBoost model with the best hyperparameters
final_xgb = xgb.XGBRegressor(**best_params_xgb)
final_xgb.fit(X_train, y_train)

# Use the XGBoost model to make predictions on the test data
y_pred_xgb = final_xgb.predict(X_test)

# Ensure that both actual and predicted values are positive for RMSLE calculation
y_test_xgb = np.maximum(y_test, 0)
y_pred_xgb = np.maximum(y_pred_xgb, 0)

# Calculate the mean squared logarithmic error (MSLE) for XGBoost
msle_xgb = mean_squared_log_error(y_test_xgb, y_pred_xgb)

# Calculate the RMSLE for XGBoost
rmsle_xgb = np.sqrt(msle_xgb)

5. Linear Regression model

In [32]:
# Continue with Linear Regression as previously shown
# ...

# Create the Linear Regression model
linear_reg = LinearRegression()

# Fit the Linear Regression model
linear_reg.fit(X_train, y_train)

# Use the Linear Regression model to make predictions on the test data
y_pred_linear = linear_reg.predict(X_test)

# Ensure that both actual and predicted values are positive for RMSLE calculation
y_test_linear = np.maximum(y_test, 0)
y_pred_linear = np.maximum(y_pred_linear, 0)

# Calculate the mean squared logarithmic error (MSLE) for Linear Regression
msle_linear = mean_squared_log_error(y_test_linear, y_pred_linear)

# Calculate the RMSLE for Linear Regression
rmsle_linear = np.sqrt(msle_linear)

In [33]:
model_dict = [
    {"Model": "Ridge Regression RMSLE", "Value": rmsle_ridge},
    {"Model": "Random Forest RMSLE", "Value": rmsle_rf},
    {"Model": "K Neighbors Regressor RMSLE", "Value": rmsle_knn},
    {"Model": "XGBoost RMSLE", "Value": rmsle_xgb},
    {"Model": "Linear Regression RMSLE", "Value": rmsle_linear}
]


Models_5 = pd.DataFrame(model_dict)
Models_5

,Model,Value
0,Ridge Regression RMSLE,0.388625
1,Random Forest RMSLE,0.345866
2,K Neighbors Regressor RMSLE,0.363678
3,XGBoost RMSLE,0.345351
4,Linear Regression RMSLE,0.389979


**Exporting Model**

In [34]:
import pickle
from sklearn.preprocessing import StandardScaler

# Initialize StandardScaler instance
scaler = StandardScaler()

# Ensure final_xgb has been trained (using the fit method) before this step
final_xgb.save_model("src/xgb_model.json")

# Generate a list of module names and their versions and write to requirements.txt
requirements = "\n".join(f"{m.__name__}={m.__version__}" for m in globals().values() if getattr(m, "__version__", None))

with open("requirements.txt", "w") as f:
    f.write(requirements)

# Create a dictionary containing the scaler and the model
export = {"scaler": scaler, "model": final_xgb}

# Use pickle to serialize and save the dictionary to a file
with open("src/streamlit_Basics.pkl", "wb") as file:
    pickle.dump(export, file)

